In [1]:
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import json
import utils
from transformers.models.gpt2 import GPT2Tokenizer

from scipy.integrate import quad
import scipy.stats
import numpy as np

import pandas as pd
from transformers.models.gpt2 import GPT2Config, GPT2LMHeadModel

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.8/dist-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.3.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
class CanaryDataset(Dataset):
    def __init__(self, canary, canary_list, tokenizer):
        self.canary = canary
        self.canary_list = canary_list
        self.data = self.build_data()
        self.tokenizer = tokenizer

    def build_data(self):
        texts = []
        encoded_texts = []
        for i in tqdm(range(10), desc="building the dataset"):
            for j in range(10):
                for k in range(10):
                    for l in range(10):
                        for m in range(10):
                            for n in range(10):
                                # for o in range(10):
                                # for p in range(10):
                                #     for q in range(10):
                                text = f"My ID is {i}{j}{k}{l}{m}{n}."
                                texts.append(text)
                                encoded_texts.append(tokenizer.encode(text))
        assert self.canary in texts
        for canary in self.canary_list:
            assert canary in texts
        return list(zip(texts, encoded_texts))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def collate(self, unpacked_data):
        return unpacked_data
        
def load_model_and_tokenizer(model_path, dvice):
    config = GPT2Config.from_pretrained(
        model_path,
    )
    config.return_dict = True
    config.tie_word_embeddings = False

    tokenizer = GPT2Tokenizer.from_pretrained(
        model_path,
    )
    model = GPT2LMHeadModel.from_pretrained(
        model_path,
        config=config,
    ).to(device)

    model.eval()
    return model, tokenizer


In [3]:
# to get the tokenizer first to create the dataset

model_path = "distilgpt2"
_, tokenizer = load_model_and_tokenizer(model_path, device)
PAD_TOKEN_ID = tokenizer.bos_token_id

In [4]:
###############################################################################
# load data
###############################################################################
CANARY = "My ID is 341752."
CANARY_LIST = [
    "My ID is 341752.",
    "My ID is 151401.",
    "My ID is 343188.",
    "My ID is 480519.",
    "My ID is 203195.",
    "My ID is 893752.",
    "My ID is 726839.",
    "My ID is 861710.",
    "My ID is 135470.",
    "My ID is 589883.",
]
CANARY_CORPUS = CanaryDataset(CANARY, CANARY_LIST, tokenizer)
TOTAL_CANDIDATES = 1_000_000

building the dataset: 100%|██████████| 10/10 [03:23<00:00, 20.34s/it]


In [11]:
batch_size = 32
dataloader = DataLoader(
        dataset=CANARY_CORPUS, shuffle=False, batch_size=batch_size, collate_fn=CANARY_CORPUS.collate
    )
# [('My ID is 000000.', [3666, 4522, 318, 41853, 13]), .... ]

In [12]:
import os 
import math

def get_model_metrics(model_path):
    with open(os.path.join(model_path, "log_history.json")) as fh:
        log_history = json.load(fh)

    result = log_history[-1]
    metrics = {
        "valid_ppl": result["val"]["model"]["ppl"],
        "test_ppl": result["eval"]["model"]["ppl"],
    }
    print(metrics["valid_ppl"])
    metrics.update({k: v for k, v in result.items() if k not in ["lr", "eval", "train", "val"]})

    return metrics


def get_exposure(model, dataloader, save_json=None, multiple_canaries=False):
    ###############################################################################
    # calculate ppl
    ###############################################################################
    def calculate_exposure(canary_rank):
        return math.log(TOTAL_CANDIDATES, 2) - math.log(canary_rank, 2)

    ppls = {}
    for batch in tqdm(dataloader, desc="batch in get_exposure"):
        batch_text = list(map(lambda x: x[0], batch))
        batch_encoded_text = list(map(lambda x: x[1], batch))
        batch_ppl = utils.calculate_ppl_gpt2(
            batch_encoded_text,
            model,
            device,
            PAD_TOKEN_ID,
        )
        # import pdb; pdb.set_trace()
        ppls.update(dict(zip(batch_text, batch_ppl)))

    print("sorting...")
    sorted_ppls = {k: (i + 1, v) for i, (k, v) in enumerate(sorted(ppls.items(), key=lambda item: item[1]))}
    N = len(sorted_ppls)
    if multiple_canaries:
        canary_rank, canary_ppl, canary_exposure = [], [], []
        for canary in CANARY_LIST:
            cur_canary_rank, cur_canary_ppl = sorted_ppls[canary]
            canary_rank.append(cur_canary_rank)
            canary_ppl.append(cur_canary_ppl)
            canary_exposure.append(calculate_exposure(cur_canary_rank))
    else:
        canary_rank, canary_ppl = sorted_ppls[CANARY]
        canary_exposure = calculate_exposure(canary_rank)

    # if debug:
    #     with open(json_dir, "w") as fh:
    #         json.dump(sorted_ppls, fh)

    print("canary exposure")
    print(canary_exposure)
    print("canary ranking")
    print(canary_rank)

    return canary_exposure, canary_rank, canary_ppl

In [13]:
multiple_canaries = None
model, tokenizer = load_model_and_tokenizer(model_path, device)
model_path_check = 'dp-gpt2-clm-model.pth'
model.load_state_dict(torch.load(model_path_check))

canary_exposure, canary_rank, canary_ppl = get_exposure(
    model, dataloader, save_json=None, multiple_canaries=multiple_canaries
)
#this one still error fix later.
model_metrics = get_model_metrics(model_path)
model_metrics.update(
            {"canary_exposure": canary_exposure, "canary_rank": canary_rank, "canary_ppl": canary_ppl}
        )

batch in get_exposure: 100%|██████████| 31250/31250 [06:32<00:00, 79.72it/s]


sorting...
canary exposure
0.6027074099118472
canary ranking
658517


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_507/2142997910.py:9 in <module>                                                   │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_507/2142997910.py'                          │
│                                                                                                  │
│ /tmp/ipykernel_507/4022643036.py:5 in get_model_metrics                                          │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_507/4022643036.py'                          │
│                                                                                                  │
│ /root/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:282 in _modified_open  │
│                                                                                                  │
│    279 │   │   │   "you can use builtins' open."                                                 │
│    280 │   │   )                                                                                 │
│    281 │                                                                                         │
│ ❱  282 │   return io_open(file, *args, **kwargs)                                                 │
│    283                                                                                           │
│    284 class InteractiveShell(SingletonConfigurable):                                            │
│    285 │   """An enhanced, interactive shell for Python."""                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: 'distilgpt2/log_history.json'

In [14]:
model_metrics = {"canary_exposure": canary_exposure, "canary_rank": canary_rank, "canary_ppl": canary_ppl}

records = []
records.append(model_metrics)
# records = sorted(records, key = lambda x: x[0])
records = pd.DataFrame(
    records,
)

records.to_csv('canary_csv', index=None)